## LangChain - Routing

In [1]:
from typing import Literal

from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_openai import AzureChatOpenAI

### Logical routing

In [2]:
class RouteQuery(BaseModel):
    """Route user query to the most relevant datasource"""
    datasource: Literal["python_docs", "js_docs", "golang_docs"] = Field(
        ..., 
        description="Given a user question choose which datasource would be the most relevant for answering their question"
    )

# LLM with function call
llm = AzureChatOpenAI(model="gpt-4o-mini",api_version="2024-12-01-preview")
structured_llm = llm.with_structured_output(RouteQuery)

# Prompt
system = """"
    You are an expert at routing question to the appropriate data source
    Based on the programming language the question is referring to, route
    it to the relevant data source.
"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}")
    ]
)

# Define router
router = prompt | structured_llm

In [3]:
question = """"
    Why doesn't the following code work:
    from langchain_core.prompts import ChatPromptTemplate

    prompt = ChatPromptTemplate.from_messages(["human", "speak in {language}"])
    prompt.invoke("french")
"""
result = router.invoke({"question": question})

In [4]:
result

RouteQuery(datasource='python_docs')

In [5]:
def choose_route(result):
    match result.datasource.lower():
        case "python_docs":
            return "chain for python_docs"
        
        case "js_docs":
            return "chain for js_docs"
        
        case "golang_docs":
            return "chain for golang_docs"

In [6]:
from langchain_core.runnables import RunnableLambda

In [7]:
full_chain = router | RunnableLambda(choose_route)

In [8]:
full_result = full_chain.invoke({"question": question})

In [9]:
full_result

'chain for python_docs'

### Semantical routing

In [10]:
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import AzureOpenAIEmbeddings

In [11]:
physics_template = """
    You are a very smart physics professor. \
    You are great at answering questions about physics in a concise and easy to understand manner. \
    When you don't know the answer to a question you admit that you don't know.

    Here is a question:
    {query}
"""

math_template = """
    You are a very good mathematician. You are great at answering math questions. \
    You are so good because you are able to break down hard problems into their component parts, \
    answer the component parts, and then put them together to answer the broader question.

    Here is a question:
    {query}
"""

In [12]:
# Embed prompts
embeddings = AzureOpenAIEmbeddings(model="text-embedding-3-large")
prompt_templates = [physics_template, math_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)

In [13]:
# Route question to prompt
def prompt_router(input):
    # Embed question
    query_embedding = embeddings.embed_query(input["query"])
    # Compute similarity
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    # chosen prompt
    print("Using MATH" if most_similar == math_template else "Using PHYSICS")
    return PromptTemplate.from_template(most_similar)

In [14]:
chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | llm
    | StrOutputParser()
)

In [15]:
chain.invoke("What is a black hole?")

Using PHYSICS


'A black hole is a region in space where the gravitational pull is so strong that nothing, not even light, can escape from it. This occurs when a massive star collapses under its own gravity at the end of its life cycle. The boundary surrounding a black hole is called the event horizon, and once an object crosses this threshold, it cannot return. Black holes can vary in size, from small "stellar" black holes formed by the collapse of individual stars to supermassive black holes, which are found at the centers of galaxies and can be millions or billions of times more massive than our Sun.'